### Implement standard binary tree/grid-based numerical algorithm for American Option Pricing and ensure it validates against Black-Scholes formula for Europeans

### Implement Longstaff-Schwartz Algorithm and ensure it validates against binary tree/grid-based solution for path-independent options